# Kream Data 크롤러
- products 목록의 제품들을 모두 크롤링한다.
- 오류 발생시 해당 제품은 체크 후 넘어간다.

In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

pd.options.display.max_rows=100
pd.options.display.max_columns=100

## 크롬드라이버 열고, kream 로그인

In [293]:
# 크롬창 열기
driver = webdriver.Chrome('./chromedriver')

# 윈도우 사용자
# driver = webdriver.Chrome('.//chromedriver.exe')

<ipython-input-293-fbec8bd2716a>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [ ]:
# 크림 홈페이지 접속 후 로그인 미리 실행
# https://kream.co.kr/

## products list 불러오기

In [2]:
products = pd.read_csv('data/products/products_adidas_yeezy.csv')

In [3]:
products

,Unnamed: 0,product,brand,category,number,wish,review
0,0,Adidas Yeezy Boost 350 V2 Beluga - Reflective,Adidas,Adidas Yeezy,46666,"4,717",231
1,1,Adidas Yeezy Boost 350 V2 Blue Tint,Adidas,Adidas Yeezy,12491,"5,641",256
2,2,Adidas Yeezy 500 Ash Grey,Adidas,Adidas Yeezy,46083,"2,282",28
3,3,Adidas Yeezy Boost 350 V2 MX Rock,Adidas,Adidas Yeezy,47158,"1,333",52
4,4,Adidas Yeezy Boost 350 V2 Israfil,Adidas,Adidas Yeezy,25011,"1,594",74
...,...,...,...,...,...,...,...
314,314,(Kids) Adidas Yeezy Boost 350 V2 Citrin,Adidas,Adidas Yeezy,22114,10,0
315,315,(Kids) Adidas Yeezy Boost 350 V2 Lundmark,Adidas,Adidas Yeezy,21255,9,0
316,316,(W) Adidas Yeezy 950 Moonrock,Adidas,Adidas Yeezy,5831,6,2
317,317,(W) Adidas Yeezy 950 Turtle Dove,Adidas,Adidas Yeezy,5829,3,0


## 크롤링 함수 만들기

In [ ]:
crawling_list = []

In [360]:
# 스크롤 다운 함수 정의
import datetime
    
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        itemlist = driver.find_element_by_class_name("price_body")
        driver.execute_script("arguments[0].scrollBy(0, -100)", itemlist)
        driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", itemlist)
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

# 크롤링 함수 (제품번호, 스크롤다운 초)
def crawler(number, seconds, re_list):
    # products 신발 선택
    url = 'https://kream.co.kr/products/{0}'.format(number)
    driver.get(url)
    # 암시적 대기, 웹페이지 전체가 뜰때까지 대기
    driver.implicitly_wait(5)
    # 이름 뽑기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    item = soup.select('div.main_title_box > p.title')
    product_name = item[0].text
    # 컬럼 뽑기
    item = soup.select('dl.detail_product')
    cols = item[0].text.replace('\n', '').replace(' ','')
    r1 = cols.find('출시일')
    r2 = cols.find('컬러')
    r3 = cols.find('발매가')
    release_date = cols[r1+3:r2]
    color = cols[r2+2:r3]
    release_price = cols[r3+3:]
    # end_date
    end_date = soup.select('td.table_td.align_right')[1].text.replace('\n','').replace(' ','')
    # 체결 내역 더보기 클릭 (copy Xpath)
    driver.find_element_by_xpath('//*[@id="panel1"]/a').click()
    # 체결 내역 날짜 오름차순 정렬
    driver.find_element_by_xpath('//*[@id="panel1"]/div/div/div[1]/div/div[3]/a').click()
    # 체결 내역 더보기 스크롤 다운
    doScrollDown(seconds)
    # 데이터 추출
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    size_list = []
    price_list = []
    date_list = []
    items = soup.select('div.body_list > div.list_txt')
    for i in range(len(items)):
        item = items[i].text.replace('\n', '').replace(' ','')
        if i%3 == 0:
            size_list.append(item)
        elif i%3 == 1:
            price_list.append(item)
        elif i%3 == 2:
            date_list.append(item)
    df = pd.DataFrame({'size': size_list, 'price': price_list, 'date': date_list})
    df['product'] = product_name
    df['release_date'] = release_date
    df['color'] = color
    df['release_price'] = release_price
    file_name = product_name.replace(' ','_')
    df.to_csv('data/transactions/{0}.csv'.format(file_name))
    if df['date'].max() == end_date:
        print('<<크롤링 완벽히 성공>>')
    else:
        print("@@@@@@@스크롤 부족!! 이 상품 다시 크롤링하세요!@@@@@@@")
    crawling_list.append([product_name, number, df['date'].min(), df['date'].max(), end_date, len(df)])
    print('{0} ~ {1} ({2})'.format(df['date'].min(), df['date'].max(), end_date))
    print(f'{file_name} 저장완료!')
    print('----'*10)

## 크롤링하기
- cralwer함수
    - number는 상품번호
    - seconds는 스크롤내리는 초 설정
- 설명


20/05/28 ~ 21/12/06 (21/12/06)<br>
Nike_x_Travis_Scott_SB_Dunk_Low_(Special_Box) 저장완료!<br>
18번째 완료

- 위와 같은 메시지가 나오는데 이걸 설명해줄게
    - 20/05/28 ~ 21/12/06 (21/12/06) 여기서 앞은 우리가 추출한 최소,최대 날짜고, (괄호)안에 들어가는거는 최대날짜와 일치해야 모든 스크롤을 잘 돌린 것임.
    - 저장이 완료되면 몇번째 상품인지도 나오니깐, 만약에 데이터가 너무 커서 n번째에 중단되었다면 다음번 실행때는 n번째부터 돌아가도록 하면됨.


In [368]:
numbers = products['number']
for i in range(len(numbers)):
    if i >= 0:
        number = numbers[i]
        crawler(number, 100)
        print(f'{i}번째 {number} 크롤링 완료')
        print('===='*10)
    else:
        continue

IndexError: list index out of range

In [ ]:
pd.DataFrame(crawling_list)

#### 참고 문헌
- https://hello-bryan.tistory.com/194,  https://jcstory94.tistory.com/69  스크롤다운 설명 링크
- https://yganalyst.github.io/pythonic/memo_16_except/, https://medium.com/@peteryun/python-selenium%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%ED%81%AC%EB%A1%A4%EB%9F%AC-%EB%A7%8C%EB%93%A4%EA%B8%B0-b055cefd1195 예외처리
- https://yeko90.tistory.com/39 셀레니움 암시적 대기